<a href="https://colab.research.google.com/github/jonsnow14/jhu-ds-specialisation/blob/master/covidmapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install geopandas
!pip install geopy

In [0]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [0]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Colaba , Mumbai, India")

In [11]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Colaba, Zone 1, Mumbai, Mumbai City, Maharashtra, 400005, India
Latitude = 18.915091, Longitude = 72.8259691


In [0]:
df = pd.read_csv("coronadata.csv")
df.head()

In [0]:
df['ADDRESS'] = df['Address1'].astype(str) + ',' + \
                df['Address3'] + ',' + \
                df['Address4'] + ',' + \
                df['Address5'] + ',' + ' Sweden'   

df.head()

In [0]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['ADDRESS'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [0]:
df['point'][0][0]

In [0]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

In [0]:
df.columns

In [0]:
df = df.drop(['Address1', 'Address3', 'Address4', 'Address5','Telefon', 'ADDRESS', 'location', 'point'], axis=1)

In [0]:
df.head()

In [0]:
df.latitude.isnull().sum()

In [0]:
df = df[pd.notnull(df["latitude"])]

In [0]:
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,

In [0]:
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [0]:
map1.save("map1.html")

In [0]:

folium_map = folium.Map(location=[59.338315,18.089960],
                        zoom_start=12,
                        tiles='CartoDB dark_matter')


FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

In [0]:
folium_map.save("map2.html")

